In [1]:
import pandas as pd
import os
import json
df = pd.read_csv('runs/results_20200914_124022/india_base/agent_outputs.csv')

In [2]:
state_names = df['state_name'].unique()
state_names

array(['gujarat', 'rajasthan', 'goa', 'sikkim', 'bihar', 'haryana',
       'tamil_nadu', 'chhattisgarh', 'meghalaya', 'andhra_pradesh',
       'puducherry', 'jharkhand', 'kerala', 'arunachal_pradesh',
       'himachal_pradesh', 'karnataka', 'maharashtra', 'west_bengal',
       'madhya_pradesh', 'assam', 'punjab', 'nagaland', 'mizoram',
       'tripura', 'uttar_pradesh'], dtype=object)

In [6]:
# --- initialize tariff df ---
urdb_df = pd.DataFrame()

# --- load sample tariff --- 
sample_tariff = pd.read_pickle(os.path.join('india','reference_data', 'sample_tariff_dict.pkl'))

# --- Load tariff .csv ---
tariffs = pd.read_csv(os.path.join('india','reference_data', 'clean_CSTEP_india_tariffs.csv'))
tariffs['per_unit'] = tariffs['per_unit'].fillna('kwh')

for rate_id in df['tariff_id'].unique():

    sector, geo = rate_id.split('#')

    # --- Lookup best tariff ---
    sector_mask = (tariffs['sector_abbr'] == sector)
    geo_mask = (tariffs['state'] == geo)
    units_mask = (tariffs['per_unit'] == 'kwh') #ignore demand charge rates for now
    t = tariffs[sector_mask & geo_mask & units_mask]

    if sector in ['res', 'com']:
        if 'LT' in t['tension'].tolist():
            t = t.loc[t['tension'] == 'LT']
        else:
            t = t.loc[t['tension'] == 'HT']
    elif sector in ['ind']:
        if 'HT' in t['tension'].tolist():
            t = t.loc[t['tension'] == 'HT']
        else:
            t = t.loc[t['tension'] == 'LT']

    assert len(t) > 0, f"No tariffs for {geo}, {sector}"
    
    # --- sort tariff in order ---
    t = t.sort_values('min_kwh', ascending=True)
        
    tariff = sample_tariff.copy()
    if len(t) > 1:
        tariff['e_levels'] = t['min_kwh'].tolist()[1:]
        tariff['e_levels'][-1] = 1e10
    else:
        tariff['e_levels'] = [1e10]

    tariff['e_prices'] = t['rate_rs_per_kwh'].tolist()

    # --- jsonify tariff and write to df ---
    tariff = json.dumps(tariff)
    tariff_row = pd.DataFrame({'tariff_id':[rate_id], 'rate_json':[tariff]})
    urdb_df = urdb_df.append(tariff_row)

# # --- Save the df to a json ---
# urdb_df.reset_index(inplace=True, drop=True)
# urdb_df.to_json(os.path.join('india_base','urdb3_rates.json'))

IndexError: list assignment index out of range

In [7]:
t

,state,sector_abbr,tension,min_kwh,max_kwh,rate_rs_per_kwh,per_unit
100,rajasthan,ind,HT,NaN,NaN,7.0,kwh


In [4]:
tariffs.loc[geo_mask & sector_mask]

,state,sector_abbr,tension,min_kwh,max_kwh,rate_rs_per_kwh,per_unit
98,rajasthan,ind,LT,0.0,500.0,6.00,kwh
99,rajasthan,ind,LT,500.0,NaN,6.45,kwh
100,rajasthan,ind,HT,NaN,NaN,7.00,kwh


In [5]:
# --- Load tariff .csv ---


tariffs

,state,sector_abbr,tension,min_kwh,max_kwh,rate_rs_per_kwh,per_unit
0,jammu_kashmir,res,LT,0.0,100.0,1.69,kwh
1,jammu_kashmir,res,LT,100.0,200.0,2.29,kwh
2,jammu_kashmir,res,LT,200.0,400.0,3.30,kwh
3,jammu_kashmir,res,HT,NaN,NaN,5.01,kwh
4,jammu_kashmir,com,LT,0.0,100.0,2.81,kwh
...,...,...,...,...,...,...,...
420,andaman_nicobar_islands,com,LT,500.0,NaN,12.00,kwh
421,andaman_nicobar_islands,com,HT,0.0,NaN,12.50,kwh
422,andaman_nicobar_islands,ind,LT,0.0,500.0,6.00,kwh
423,andaman_nicobar_islands,ind,LT,500.0,NaN,8.00,kwh
